### Основные операторы PostgreSQL.

#### Задание ####

1) Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов.
2) Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно.
3) Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.
4) Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services.
5) Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'.
6) Вывести всех клиентов, у которых нет транзакций.
7) Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью.
8) Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'.

Так как в первом домашнем задании мы уже работали с этими данными и улучшили их структуру, я буду использовать созданную мной схему, а не исходную. Это кажется мне более логичным и правильным решением.

![title](Схема_0.png)

Поскольку некоторые задания имеют неоднозначную формулировку, я буду предварительно описывать, как именно я их интерпретировал.

#### 1. Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов. ####

В этом задании указано вывести только столбец с уникальными брендами, чья стоимость превышает 1500$. Цена не выводится, так как это не требуется по условию.

~~~~sql
SELECT DISTINCT brand
	FROM products
	WHERE standard_cost > 1500;
~~~~

![title](Схема_1.png)

#### 2. Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно. ####

Какие данные относятся к транзакции, я определил ещё в первом домашнем задании, поместив их в таблицу **transactions**. Это следующие столбцы: 
- transaction_id PK (Уникальный id транзакции), 
- customer_id FK (id клиента, который совершает транзакцию), 
- product_new_id FK (id продукта, который покупает клиент), 
- transaction_date (дата транзакции), 
- online_order (признак того была ли проведена транзакция онлайн), 
- order_status (статус заказа), 
- list_price (цена продажи).

Поэтому я выберу именно их, с учётом ограничений: подтверждение транзакции и период с 1 по 9 апреля 2017 года включительно.

~~~~sql
SELECT * 
	FROM transactions
	WHERE (transaction_date >= '2017-04-01') 
		AND (transaction_date <= '2017-04-09')
		AND (order_status = 'Approved');
~~~~

![title](Схема_2.png)

#### 3. Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'. ####

В этом задании указано вывести только столбец с профессиями, которые начинаются с фразы 'Senior' из сферы IT или Financial Service. Само название сферы не выводится, так как это не требуется по условию. Если профессия пренадлежит и к одной и другой сфере, она выводится один раз, не дублируясь.

~~~~sql
SELECT DISTINCT job_title
	FROM job_categories
	WHERE (job_title LIKE 'Senior%') 
		AND ((job_industry_category = 'IT') OR (job_industry_category = 'Financial Services'));
~~~~

![title](Схема_3.png)

#### 4. Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services. ####

В этом задании указано вывести только столбец с брендами, которые закупают клиенты из сферы Financial Service.

~~~~sql
SELECT DISTINCT brand
    FROM products
        INNER JOIN transactions ON products.product_new_id = transactions.product_new_id
        INNER JOIN customers ON customers.customer_id = transactions.customer_id
        INNER JOIN job_categories ON job_categories.job_id = customers.job_id
    WHERE job_categories.job_industry_category = 'Financial Services';
~~~~

![title](Схема_4.png)

#### 5. Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'. ####

Какие данные относятся к характеристике клиента, я определил ещё в первом домашнем задании, поместив их в таблицу **customers**. Это следующие столбцы: 
- customer_id PK (Уникальный id клиента)
- first_name (Имя)
- last_name (Фамилия)
- gender (пол)
- DOB (дата рождения)
- wealth_segment (сегмент благосостояния)
- deceased_indicator (флаг, жив клиент или мертв)
- own_car (владеет ли машиной)
- address_id FK (Уникальный id адреса)
- job_id FK (Уникальный id должности)

В этом задании не совсем ясно, нужно ли вывести клиентов, закупавших продукцию всех трёх брендов или хотя бы одного из них. Я решил, что требуется найти клиентов, совершавших онлайн-покупки у всех трёх брендов. Выполнил это и вывел первых 10 таких клиентов.

~~~~sql
SELECT DISTINCT customers.*
	FROM products
		INNER JOIN transactions ON products.product_new_id = transactions.product_new_id
		INNER JOIN customers ON customers.customer_id = transactions.customer_id
	WHERE brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
		AND (transactions.online_order = TRUE)
	GROUP BY customers.customer_id
	HAVING COUNT(DISTINCT brand) = 3
	LIMIT 10;
~~~~

![title](Схема_5.png)

#### 6. Вывести всех клиентов, у которых нет транзакций. ####

Какие данные относятся к характеристике клиента, я определил ещё в первом домашнем задании, поместив их в таблицу **customers**. Это следующие столбцы: 
- customer_id PK (Уникальный id клиента)
- first_name (Имя)
- last_name (Фамилия)
- gender (пол)
- DOB (дата рождения)
- wealth_segment (сегмент благосостояния)
- deceased_indicator (флаг, жив клиент или мертв)
- own_car (владеет ли машиной)
- address_id FK (Уникальный id адреса)
- job_id FK (Уникальный id должности)

Требуется найти таких клиентов, которые не упоминаются в таблице **transactions**.

~~~~sql
SELECT customers.*
	FROM customers
		LEFT JOIN transactions ON customers.customer_id = transactions.customer_id
		WHERE transactions.customer_id IS NULL
~~~~

![title](Схема_6.png)

#### 7. Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью. ####

Какие данные относятся к характеристике клиента, я определил ещё в первом домашнем задании, поместив их в таблицу **customers**. Это следующие столбцы: 
- customer_id PK (Уникальный id клиента)
- first_name (Имя)
- last_name (Фамилия)
- gender (пол)
- DOB (дата рождения)
- wealth_segment (сегмент благосостояния)
- deceased_indicator (флаг, жив клиент или мертв)
- own_car (владеет ли машиной)
- address_id FK (Уникальный id адреса)
- job_id FK (Уникальный id должности)


Требуется найти таких клиентов из сферы IT, которые покупали продукт с самой большой себестоймостю.

~~~~sql
SELECT DISTINCT customers.*
	FROM customers
		INNER JOIN transactions ON customers.customer_id = transactions.customer_id
		INNER JOIN job_categories ON customers.job_id = job_categories.job_id
		INNER JOIN products ON products.product_new_id = transactions.product_new_id
	WHERE (job_categories.job_industry_category = 'IT') 
		AND products.standard_cost = (
			SELECT MAX(standard_cost) FROM products
		);
~~~~

![title](Схема_7.png)

#### 8. Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'. ####

Какие данные относятся к характеристике клиента, я определил ещё в первом домашнем задании, поместив их в таблицу **customers**. Это следующие столбцы: 
- customer_id PK (Уникальный id клиента)
- first_name (Имя)
- last_name (Фамилия)
- gender (пол)
- DOB (дата рождения)
- wealth_segment (сегмент благосостояния)
- deceased_indicator (флаг, жив клиент или мертв)
- own_car (владеет ли машиной)
- address_id FK (Уникальный id адреса)
- job_id FK (Уникальный id должности)


Требуется найти таких клиентов из сферы IT или Health, которые совершали подтвержденные транзакции с 7 июля 2017 по 17 июля 2017. Я сделал включительно, поскольку мне показалось это логичным, хоть уточнения и не было.

![title](Схема_8.png)